In [1]:
import pandas as pd

df = pd.read_csv('weather.csv')
print(df)

            date  precipitation  temp_max  temp_min  wind  weather
0     2012-01-01            0.0      12.8       5.0   4.7  drizzle
1     2012-01-02           10.9      10.6       2.8   4.5     rain
2     2012-01-03            0.8      11.7       7.2   2.3     rain
3     2012-01-04           20.3      12.2       5.6   4.7     rain
4     2012-01-05            1.3       8.9       2.8   6.1     rain
...          ...            ...       ...       ...   ...      ...
1456  2015-12-27            8.6       4.4       1.7   2.9     rain
1457  2015-12-28            1.5       5.0       1.7   1.3     rain
1458  2015-12-29            0.0       7.2       0.6   2.6      fog
1459  2015-12-30            0.0       5.6      -1.0   3.4      sun
1460  2015-12-31            0.0       5.6      -2.1   3.5      sun

[1461 rows x 6 columns]


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           1461 non-null   object 
 1   precipitation  1461 non-null   float64
 2   temp_max       1461 non-null   float64
 3   temp_min       1461 non-null   float64
 4   wind           1461 non-null   float64
 5   weather        1461 non-null   object 
dtypes: float64(4), object(2)
memory usage: 68.6+ KB


In [3]:
df['weather'].unique()

array(['drizzle', 'rain', 'sun', 'snow', 'fog'], dtype=object)

In [4]:
df['weather'].value_counts()

weather
rain       641
sun        640
fog        101
drizzle     53
snow        26
Name: count, dtype: int64

In [15]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

#  Label encoding the target column
label_encoder = LabelEncoder()
df['weather'] = label_encoder.fit_transform(df['weather'])
print(df)

# Scaling the independent features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df[['precipitation', 'temp_max', 'temp_min', 'wind']])
print(scaled_features)

            date  precipitation  temp_max  temp_min  wind  weather
0     2012-01-01            0.0      12.8       5.0   4.7        0
1     2012-01-02           10.9      10.6       2.8   4.5        2
2     2012-01-03            0.8      11.7       7.2   2.3        2
3     2012-01-04           20.3      12.2       5.6   4.7        2
4     2012-01-05            1.3       8.9       2.8   6.1        2
...          ...            ...       ...       ...   ...      ...
1456  2015-12-27            8.6       4.4       1.7   2.9        2
1457  2015-12-28            1.5       5.0       1.7   1.3        2
1458  2015-12-29            0.0       7.2       0.6   2.6        1
1459  2015-12-30            0.0       5.6      -1.0   3.4        4
1460  2015-12-31            0.0       5.6      -2.1   3.5        4

[1461 rows x 6 columns]
[[0.         0.38709677 0.47637795 0.47252747]
 [0.19499106 0.32795699 0.38976378 0.45054945]
 [0.01431127 0.35752688 0.56299213 0.20879121]
 ...
 [0.         0.23655914 0

In [8]:
type(scaled_features)

numpy.ndarray

In [9]:
scaled_features.ndim

2

In [13]:
df['weather'].ndim

1

In [20]:
import numpy as np

# defining a new function to create seq from features
def create_sequences(features, labels, time_steps=3):
    X, y = [], []
    for i in range(len(features) - time_steps):
        X.append(features[i:i + time_steps])
        y.append(labels[i + time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_features, df['weather'])

print(X)
print(y)

[[[0.         0.38709677 0.47637795 0.47252747]
  [0.19499106 0.32795699 0.38976378 0.45054945]
  [0.01431127 0.35752688 0.56299213 0.20879121]]

 [[0.19499106 0.32795699 0.38976378 0.45054945]
  [0.01431127 0.35752688 0.56299213 0.20879121]
  [0.36314848 0.37096774 0.5        0.47252747]]

 [[0.01431127 0.35752688 0.56299213 0.20879121]
  [0.36314848 0.37096774 0.5        0.47252747]
  [0.02325581 0.28225806 0.38976378 0.62637363]]

 ...

 [[0.         0.16129032 0.27952756 0.23076923]
  [0.15384615 0.16129032 0.34645669 0.27472527]
  [0.02683363 0.17741935 0.34645669 0.0989011 ]]

 [[0.15384615 0.16129032 0.34645669 0.27472527]
  [0.02683363 0.17741935 0.34645669 0.0989011 ]
  [0.         0.23655914 0.30314961 0.24175824]]

 [[0.02683363 0.17741935 0.34645669 0.0989011 ]
  [0.         0.23655914 0.30314961 0.24175824]
  [0.         0.19354839 0.24015748 0.32967033]]]
[2 2 2 ... 1 4 4]


In [21]:
X.ndim

3

In [22]:
y.ndim

1

In [24]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Assuming X and y are already preprocessed and scaled
# X.shape: (samples, time_steps, features)
# y.shape: (samples,) -> categorical labels

# Convert y to one-hot encoding if it's classification
y_cat = to_categorical(df['weather'])
X_train, X_test, y_train, y_test = train_test_split(scaled_features, y_cat, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_train.shape[0])
print(X_train.shape[1])

# Reshape for LSTM [samples, time_steps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
print(X_train.shape)

X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

(1168, 4)
1168
4
(1168, 4, 1)


In [25]:
# preparing the model
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    LSTM(64, return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(y_train.shape[1], activation='softmax')  # Softmax for multi-class classification
])

# compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Training the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32,
                    validation_data=(X_test, y_test), verbose=1)

C:\Users\Gaurav\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 4, 128)         │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,213 (461.77 KB)

 Trainable params: 118,213 (461.77 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.4255 - loss: 1.3474 - val_accuracy: 0.4471 - val_loss: 1.1760
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4452 - loss: 1.1293 - val_accuracy: 0.4369 - val_loss: 1.1398
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4238 - loss: 1.1093 - val_accuracy: 0.4232 - val_loss: 1.1665
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4426 - loss: 1.1016 - val_accuracy: 0.4096 - val_loss: 1.1247
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4426 - loss: 1.1012 - val_accuracy: 0.4471 - val_loss: 1.1224
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4401 - loss: 1.1066 - val_accuracy: 0.4471 - val_loss: 1.1339
Epoch 7/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4580 - loss: 1.1029 - val_accuracy: 0.5870 - val_loss: 1.1223
Epoch 8/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4435 - loss: 1.1005 - val_accuracy: 0.4232 - val_loss

In [26]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.2f}')

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8294 - loss: 0.5539 
Test Accuracy: 0.83


In [27]:
# Example: Predict weather for [precipitation=0.1, temp_max=25, temp_min=18, wind=3]
sample = np.array([[0.1, 25, 18, 3]])
sample_scaled = scaler.transform(sample).reshape(1, 1, 4)

prediction = model.predict(sample_scaled)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])
print("Predicted weather:", predicted_label[0])

C:\Users\Gaurav\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


ValueError: Exception encountered when calling LSTMCell.call().

[1mDimensions must be equal, but are 4 and 1 for '{{node sequential_1/lstm_1/lstm_cell_1/MatMul}} = MatMul[T=DT_FLOAT, grad_a=false, grad_b=false, transpose_a=false, transpose_b=false](sequential_1/lstm_1/strided_slice_1, sequential_1/lstm_1/lstm_cell_1/Cast/ReadVariableOp)' with input shapes: [1,4], [1,512].[0m

Arguments received by LSTMCell.call():
  • inputs=tf.Tensor(shape=(1, 4), dtype=float32)
  • states=('tf.Tensor(shape=(1, 128), dtype=float32)', 'tf.Tensor(shape=(1, 128), dtype=float32)')
  • training=False